# Tokopedia

In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# Input URL toko
url = input("Masukkan URL toko: ")

# Input jumlah halaman
try:
    total_pages = int(input("Masukkan jumlah halaman yang ingin discan: "))
except ValueError:
    print("Input jumlah halaman tidak valid, gunakan angka.")
    total_pages = 1  # Default fallback

time.sleep(6)

if url:
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    no = 1
    data = []

    for i in range(total_pages):
        soup = BeautifulSoup(driver.page_source, "html.parser")
        containers = soup.findAll('article', attrs={'class': 'css-1pr2lii'})

        for container in containers:
            try:
                username_tag = container.find('span', attrs={'class': 'name'})
                username = username_tag.text.strip() if username_tag else ("Null")
                50
                product_name_tag = container.find('p', attrs={'class': 'css-1mjoy75-unf-heading e1qvo2ff8'})
                product_name = product_name_tag.text.strip() if product_name_tag else ("Null")

                variant_tag = container.find('p', attrs={'class': 'css-aajcjd-unf-heading e1qvo2ff8'})
                variant = variant_tag.text.strip() if variant_tag else ("Null")

                product_link_tag = container.find('a', attrs={'class': 'styProduct'})
                product_link = product_link_tag['href'] if product_link_tag else ("Null")

                review_tag = container.find('span', attrs={'data-testid': 'lblItemUlasan'})
                review = review_tag.text.strip() if review_tag else ("belum ada ulasan")

                rating_container = container.find('div', attrs={'data-testid': 'icnStarRating'})
                if rating_container and 'aria-label' in rating_container.attrs:
                    rating_label = rating_container['aria-label']
                    rating = int(''.join(filter(str.isdigit, rating_label)))
                else:
                    rating = None

                raw_date = container.find('p', class_='css-vqrjg4-unf-heading e1qvo2ff8').text.strip().lower()

                if raw_date == 'hari ini':
                    date = datetime.now().strftime('%Y-%m-%d')
                elif raw_date == 'kemarin':
                    date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
                else:
                    try:
                        if 'lebih dari' in raw_date:
                            if 'tahun' in raw_date:
                                date = (datetime.now() - relativedelta(years=2)).strftime('%Y-%m-%d')
                            elif 'bulan' in raw_date:
                                date = (datetime.now() - relativedelta(months=2)).strftime('%Y-%m-%d')
                            elif 'minggu' in raw_date:
                                date = (datetime.now() - timedelta(weeks=3)).strftime('%Y-%m-%d')
                            else:
                                date = None

                        elif 'tahun' in raw_date:
                            years_ago = int(raw_date.split()[0])
                            date = (datetime.now() - relativedelta(years=years_ago)).strftime('%Y-%m-%d')
                        elif 'bulan' in raw_date:
                            months_ago = int(raw_date.split()[0])
                            date = (datetime.now() - relativedelta(months=months_ago)).strftime('%Y-%m-%d')
                        elif 'minggu' in raw_date:
                            weeks_ago = int(raw_date.split()[0])
                            date = (datetime.now() - timedelta(weeks=weeks_ago)).strftime('%Y-%m-%d')
                        elif 'hari' in raw_date:
                            days_ago = int(raw_date.split()[0])
                            date = (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
                        else:
                            # Coba parsing tanggal literal, seperti "2 Jan"
                            date_obj = datetime.strptime(raw_date + f" {datetime.now().year}", '%d %b %Y')
                            date = date_obj.strftime('%Y-%m-%d')
                    except Exception as e:
                        date = None
                        print(f"Gagal parse tanggal: {raw_date} | Error: {e}")

                kendala_tag = container.find('p', class_='css-1129ssl-unf-heading e1qvo2ff8')
                kendala = None

                if kendala_tag:
                    kendala_text = kendala_tag.text.strip()
                    if "kendala:" in kendala_text.lower():
                        kendala = kendala_text.split(":", 1)[1].strip()
                else:
                    if any(word in review.lower() for word in ['rusak', 'cacat', 'tidak berfungsi', 'telat', 'salah', 'komplain', 'keluhan', 'lambat']):
                        kendala = review

                data.append({
                    "No": no,
                    "Nama User": username,
                    "Nama Produk": product_name,
                    "Varian Produk": variant,
                    "Link Produk": product_link,
                    "Rating": rating,
                    "Tanggal": date,
                    "Ulasan": review,
                })

                no += 1

            except Exception as e:
                print("Error parsing container:", e)
                continue

        # Klik halaman berikutnya
        try:
            time.sleep(1)
            next_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label^='Laman berikutnya']")
            next_button.click()
            time.sleep(1)
        except Exception as e:
            print("Tidak bisa klik halaman berikutnya:", e)
            break

    driver.quit()

    # Simpan ke CSV
    df = pd.DataFrame(data)
    df.to_csv("Tokopedia.csv", index=False)
    print("Scraping selesai. Data disimpan di Tokopedia.csv")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_15628\3062601977.py:32: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  containers = soup.findAll('article', attrs={'class': 'css-1pr2lii'})


Scraping selesai. Data disimpan di Tokopedia.csv
